# Documentation and resources

**Big Query**
- Colab has an example notebook on BigQuery too.  File > Open notebook > Examples > Getting Started with BigQuery.
- Also a Big Query Snippets Example Notebook
- [BigQuery Documentation]( https://cloud.google.com/bigquery/docs )
- [Open Data Sets]( https://console.cloud.google.com/marketplace/browse?filter=solution-type:dataset )
- [Reddit - list of data sets available on BQ]( https://www.reddit.com/r/bigquery/wiki/datasets )


**Big Query Console**  
- [Google Cloud Console]( https://console.cloud.google.com )
- Make sure your project is selected
- Scroll down to BigQuery on the left menu
- [Setup and query instructions]( https://cloud.google.com/bigquery/docs/quickstarts/query-public-dataset-console )

**SQL**
- [Kaggle Intro to SQL]( https://www.kaggle.com/learn/intro-to-sql ) uses BigQuery
- [Kaggle Advanced SQL]( https://www.kaggle.com/learn/advanced-sql )

#  Linking BigQuery to Colab

## Getting started

**You will only need to do this part once.**

1. Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.

    - Create Project
    - Project Name
    - Location

2. [Enable BigQuery APIs](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) for the project

**Note:** You get 1 TB/month of free queries for open datasets
- Kaggle gives you 5 TB/month free



## Imports

In [13]:
# pandas module
import pandas as pd
import pandas_gbq

# gcp modules
from google.cloud import bigquery

# colab modules
from google.colab import auth
from google.colab import syntax
from google.colab import userdata


### Provide your credentials

In [14]:
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [15]:
%load_ext google.colab.data_table
# %unload_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

## List projects



In order to query BigQuery, you will need to specify a project ID.  To get a list of project IDs associated with your account, run the following command.

In [16]:
%alias gcloud gcloud

In [17]:
gcloud projects list --sort-by=projectId

PROJECT_ID                     NAME                    PROJECT_NUMBER
data-science-project-3-439515  Data-Science-project-3  428650296072


## Declare the Cloud project ID which will be used throughout this notebook

To set up secret open keys tab on colab and provide projectid as the value and name your key to use your project from google colab

In [23]:
project_id = userdata.get('proj-3')

In [24]:
project_id


'data-science-project-3-439515'

## Samples data set



The [GSOD table](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=gsod&page=table) in the Samples data set contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


# Use BigQuery via magics



The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [25]:
# Display query output immediately

%%bigquery --project {project_id}
SELECT
  COUNT(1) as total_rows
FROM `bigquery-public-data.samples.gsod`

Query is running:   0%|          |

Downloading:   0%|          |

,total_rows
0,114420316


In [26]:
# Save output in a variable `df`

%%bigquery df --project {project_id}
SELECT
  COUNT(1) as total_rows
FROM `bigquery-public-data.samples.gsod`

Query is running:   0%|          |

Downloading:   0%|          |

In [27]:
df

,total_rows
0,114420316


In [28]:
f'{df.iloc[0,0]:_}'

'114_420_316'

# Use BigQuery through google-cloud-bigquery



See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).


## Sample approximately 2000 random rows

### Count total number of rows

In [29]:
client = bigquery.Client(project=project_id)

row_count = client.query('''
  SELECT
    COUNT(1) as total
  FROM `bigquery-public-data.samples.gsod`
  '''
).to_dataframe()["total"][0]

print(f'Full dataset has {row_count:_} rows')


Full dataset has 114_420_316 rows


### Describe the sampled data

In [30]:
sample_count = 2000
df = client.query(f'''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < {sample_count}/{row_count}
''').to_dataframe()


In [31]:
df.describe().transpose().astype({"count": int})

,count,mean,std,min,25%,50%,75%,max
station_number,2063,493971.275812,300800.537625,10014.0,230370.0,489200.0,724602.5,999999.0
wban_number,2063,90966.826951,25571.378349,13.0,99999.0,99999.0,99999.0,99999.0
year,2063,1986.759089,16.065688,1933.0,1977.0,1989.0,2000.0,2010.0
month,2063,6.591372,3.452218,1.0,4.0,7.0,10.0,12.0
day,2063,15.836646,8.838346,1.0,8.0,16.0,23.0,31.0
mean_temp,2063,52.128163,23.609143,-54.900002,38.5,54.900002,69.900002,100.900002
num_mean_temp_samples,2063,12.807077,7.843004,4.0,7.0,8.0,23.0,24.0
mean_dew_point,1972,41.618966,21.993648,-59.599998,29.700001,44.200001,56.624999,80.599998
num_mean_dew_point_samples,1972,12.704361,7.827563,4.0,7.0,8.0,23.0,24.0
mean_sealevel_pressure,1561,1015.109993,9.535269,951.700012,1009.799988,1015.0,1020.599976,1059.199951


### View the first 10 rows

In [32]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,725660,24028,1943,5,21,56.200001,24,44.400002,24,1017.900024,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
1,63800,99999,1949,10,5,57.400002,8,51.400002,8,NaN,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
2,719250,99999,1950,1,8,-23.299999,8,-29.400000,8,1002.000000,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
3,203570,99999,1951,10,19,26.500000,4,18.500000,4,993.099976,...,NaN,<NA>,0.02,NaN,False,False,False,False,False,False
4,276480,99999,1952,1,7,15.000000,4,8.500000,4,1026.300049,...,NaN,<NA>,0.12,NaN,True,True,True,True,True,True
5,999999,34066,1961,11,1,47.099998,13,41.599998,13,1030.199951,...,NaN,<NA>,0.00,NaN,True,True,True,True,True,True
6,943800,99999,1965,11,8,79.000000,7,71.099998,7,NaN,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
7,161700,99999,1967,10,31,52.400002,7,48.700001,7,1012.900024,...,NaN,<NA>,0.31,NaN,False,False,False,False,False,False
8,727575,94925,1967,10,28,23.600000,24,14.400000,24,1013.099976,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
9,336640,99999,1977,7,9,68.599998,6,61.400002,6,1006.799988,...,NaN,<NA>,0.04,NaN,False,False,False,False,False,False


In [33]:
df.shape

(2063, 31)

In [34]:
df.isnull().sum()

,0
station_number,0
wban_number,0
year,0
month,0
day,0
mean_temp,0
num_mean_temp_samples,0
mean_dew_point,91
num_mean_dew_point_samples,91
mean_sealevel_pressure,502


In [35]:
# 10 highest total_precipitation samples
(
df
  .sort_values('total_precipitation', ascending=False)
  .head(10)
  [['station_number', 'year', 'month', 'day', 'total_precipitation']]
)


,station_number,year,month,day,total_precipitation
1084,984260,1990,9,7,6.97
1859,983280,1978,7,27,5.51
611,474300,1993,11,14,2.64
269,725109,2006,9,15,2.56
326,914130,1990,6,24,2.18
1787,913340,1981,1,4,2.05
1639,723405,1998,2,11,2.01
172,854170,1974,12,30,2.01
2033,785010,1988,10,31,1.97
1580,636120,1977,11,3,1.73


# Use BigQuery through pandas-gbq



The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation]( https://pandas-gbq.readthedocs.io/en/latest/reading.html )

In [36]:
df = pandas_gbq.read_gbq('''
  SELECT
    name, SUM(number) as count
  FROM
    bigquery-public-data.usa_names.usa_1910_2013
  WHERE
    state = 'TX'
  GROUP BY
    name
  ORDER BY
    count DESC
  LIMIT
    100
  ''', project_id=project_id, dialect='standard'
)

df.head()

Downloading: 100%|██████████|


,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [37]:
query = syntax.sql('''
  SELECT
    COUNT(1) as total_rows
  FROM
    `bigquery-public-data.samples.gsod`
''')

pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


,total_rows
0,114420316


In [38]:
type(query)

str